In [8]:
# ==========================================================
# Ensemble of EfficientNetV2-S + ShuffleNetV2 + MobileNetV3
# CIFAR-10 | PyTorch | 5 Random Seeds | 4 Epochs Each | GPU Supported
# ==========================================================

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import random
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [9]:
# --------------------------
# Device setup
# --------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [10]:
# --------------------------
# Transformations and Dataloaders
# --------------------------
transform = transforms.Compose([
    transforms.Resize((160, 160)),   # required for all three models
     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

In [11]:
# --------------------------
# Ensemble Model Definition
# --------------------------
class EnsembleNet(nn.Module):
    def __init__(self):
        super(EnsembleNet, self).__init__()
        # Load pretrained models
        self.effnet = torchvision.models.efficientnet_v2_s(weights="IMAGENET1K_V1")
        self.shufflenet = torchvision.models.shufflenet_v2_x1_0(weights="IMAGENET1K_V1")
        self.mobilenet = torchvision.models.mobilenet_v3_small(weights="IMAGENET1K_V1")

        # Modify classifiers for 10 classes (CIFAR-10)
        self.effnet.classifier[1] = nn.Linear(self.effnet.classifier[1].in_features, 10)
        self.shufflenet.fc = nn.Linear(self.shufflenet.fc.in_features, 10)
        self.mobilenet.classifier[3] = nn.Linear(self.mobilenet.classifier[3].in_features, 10)

    def forward(self, x):
        x1 = self.effnet(x)
        x2 = self.shufflenet(x)
        x3 = self.mobilenet(x)
        # Average the logits from each model
        return (x1 + x2 + x3) / 3

In [12]:
# --------------------------
# Training and Evaluation Functions
# --------------------------
def train_model(model, trainloader, criterion, optimizer, epochs=4):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}] - Loss: {running_loss/len(trainloader):.4f}")

def evaluate_model(model, testloader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    return acc, prec, rec, f1

In [13]:
# --------------------------
# 5-Seed Training & Evaluation
# --------------------------
seeds = [42, 123, 340, 777, 999]
results = []

for seed in seeds:
    print(f"\n=== Running for seed: {seed} (4 epochs) ===")
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    model = EnsembleNet().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    train_model(model, trainloader, criterion, optimizer, epochs=4)
    acc, prec, rec, f1 = evaluate_model(model, testloader)
    results.append([acc, prec, rec, f1])
    print(f"Seed {seed} → Acc: {acc:.4f}, Prec: {prec:.4f}, Rec: {rec:.4f}, F1: {f1:.4f}")



=== Running for seed: 42 (4 epochs) ===
Epoch [1/4] - Loss: 0.2979
Epoch [2/4] - Loss: 0.0725
Epoch [3/4] - Loss: 0.0373
Epoch [4/4] - Loss: 0.0306
Seed 42 → Acc: 0.9684, Prec: 0.9687, Rec: 0.9684, F1: 0.9684

=== Running for seed: 123 (4 epochs) ===
Epoch [1/4] - Loss: 0.3068
Epoch [2/4] - Loss: 0.0705
Epoch [3/4] - Loss: 0.0380
Epoch [4/4] - Loss: 0.0272
Seed 123 → Acc: 0.9730, Prec: 0.9731, Rec: 0.9730, F1: 0.9730

=== Running for seed: 340 (4 epochs) ===
Epoch [1/4] - Loss: 0.3110
Epoch [2/4] - Loss: 0.0698
Epoch [3/4] - Loss: 0.0397
Epoch [4/4] - Loss: 0.0277
Seed 340 → Acc: 0.9696, Prec: 0.9699, Rec: 0.9696, F1: 0.9696

=== Running for seed: 777 (4 epochs) ===
Epoch [1/4] - Loss: 0.3150
Epoch [2/4] - Loss: 0.0727
Epoch [3/4] - Loss: 0.0385
Epoch [4/4] - Loss: 0.0286
Seed 777 → Acc: 0.9704, Prec: 0.9706, Rec: 0.9704, F1: 0.9703

=== Running for seed: 999 (4 epochs) ===
Epoch [1/4] - Loss: 0.3083
Epoch [2/4] - Loss: 0.0659
Epoch [3/4] - Loss: 0.0395
Epoch [4/4] - Loss: 0.0304
Seed

In [14]:
# --------------------------
# Mean ± Standard Deviation
# --------------------------
results = np.array(results)
metrics = ["Accuracy", "Precision", "Recall", "F1-Score"]

print("\n=== Final Results (Mean ± SD across 5 seeds) ===")
for i, metric in enumerate(metrics):
    mean_val = results[:, i].mean()
    sd_val = results[:, i].std()
    print(f"{metric}: {mean_val:.4f} ± {sd_val:.4f}")


=== Final Results (Mean ± SD across 5 seeds) ===
Accuracy: 0.9698 ± 0.0018
Precision: 0.9701 ± 0.0018
Recall: 0.9698 ± 0.0018
F1-Score: 0.9698 ± 0.0018
